In [48]:
%load_ext autoreload
%autoreload 2

[autoreload of plaid.denoisers failed: Traceback (most recent call last):
  File "/homefs/home/lux70/micromamba/envs/plaid/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/homefs/home/lux70/micromamba/envs/plaid/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/homefs/home/lux70/micromamba/envs/plaid/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/homefs/home/lux70/code/plaid/src/plaid/denoisers/__init__.py", line 4, in <module>
    from .modules import DenoiserKwargs
ImportError: cannot import name 'DenoiserKwargs' from 'plaid.denoisers.modules' (/homefs/home/lu

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load EMA weights

In [23]:
from pathlib import Path
from omegaconf import OmegaConf

import torch
from plaid.diffusion import FunctionOrganismDiffusion
from plaid.denoisers import FunctionOrganismDiT, DenoiserKwargs
from plaid.constants import COMPRESSION_INPUT_DIMENSIONS, COMPRESSION_SHORTEN_FACTORS

device = torch.device("cuda")

In [24]:
model_id = "4hdab8dn"

ckpt_dir = Path("/data/lux70/plaid/checkpoints/plaid-compositional") 
model_path = ckpt_dir / model_id / "last.ckpt"
config_path = ckpt_dir / model_id / "config.yaml"

cfg = OmegaConf.load(config_path)

In [25]:
compression_model_id = cfg['compression_model_id']
shorten_factor = COMPRESSION_SHORTEN_FACTORS[compression_model_id]
input_dim = COMPRESSION_INPUT_DIMENSIONS[compression_model_id]

In [26]:
denoiser_kwargs = cfg.denoiser
denoiser_kwargs.pop("_target_")
denoiser = FunctionOrganismDiT(**denoiser_kwargs, input_dim=input_dim)

In [27]:
# lask.ckpt automatically links to the EMA

ckpt = torch.load(model_path)
# ckpt.keys()

In [28]:
mod_state_dict = {}
for k, v in ckpt['state_dict'].items():
    if k[:16] == "model._orig_mod.":
        mod_state_dict[k[16:]] = v

In [29]:
denoiser.load_state_dict(mod_state_dict)

<All keys matched successfully>

In [30]:
diffusion_kwargs = cfg.diffusion
diffusion_kwargs.pop("_target_")

# diffusion_kwargs['beta_scheduler_name'] = "sigmoid"
diffusion_kwargs['sampling_timesteps'] = 500

diffusion = FunctionOrganismDiffusion(model=denoiser,**diffusion_kwargs)

In [31]:
from cheap.pretrained import load_model_from_id
cheap_model = load_model_from_id(compression_model_id)
_ = cheap_model.to(device)

from cheap.proteins import LatentToSequence,LatentToStructure
latent_to_sequence = LatentToSequence()
latent_to_sequence.to(device)

latent_to_structure = LatentToStructure()
latent_to_structure.to(device)

Using tanh layer at bottleneck...
Finished loading HPCT model with shorten factor 2 and 32 channel dimensions.


Creating ESMFold...
ESMFold model loaded in 35.30 seconds.


# Sample
Human, carbohydrate metabolic process:

In [32]:
# organism_idx = org_df[org_df.organism_id == "HUMAN"].organism_index.iloc[0]
# function_idx = go_df[go_df.GO_term == "carbohydrate metabolic process"].GO_idx.iloc[0]
# print(organism_idx, function_idx)

organism_idx = 1326
function_idx = 55

In [33]:
device = torch.device("cuda")
diffusion = diffusion.to(device)

In [34]:
N, L = 32, 64
shape = (N, L, input_dim)
organism_y_idxs = torch.full((N,), organism_idx)
function_y_idxs = torch.full((N,), function_idx)

cond_scale = 2

diffusion.sampling_timesteps=1000
diffusion
sampled_latent = diffusion.ddim_sample_loop(shape, organism_idx, function_idx, return_all_timesteps=True, cond_scale=cond_scale)

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

In [35]:
print(sampled_latent.shape)
final_sample = sampled_latent[:, -1, :, :]
print(final_sample.shape)

print(final_sample.max(), final_sample.min())

torch.Size([32, 1001, 64, 32])
torch.Size([32, 64, 32])
tensor(0.9454, device='cuda:0') tensor(-0.9584, device='cuda:0')


In [36]:
sampled_uncompressed = cheap_model.decode(final_sample, downsampled_mask=None)
print(sampled_uncompressed.min(), sampled_uncompressed.max())

tensor(-0.9198, device='cuda:0') tensor(0.9899, device='cuda:0')


In [37]:
# import pickle as pkl
# with open("test_sample.pkl", "wb") as f:
#     pkl.dump(sampled_uncompressed,f)

In [38]:
from cheap.utils import LatentScaler
latent_scaler = LatentScaler()
sampled_unscaled = latent_scaler.unscale(sampled_uncompressed) 

In [39]:
print(sampled_unscaled.shape)
print(sampled_unscaled.max(), sampled_unscaled.min())

torch.Size([32, 128, 1024])
tensor(2762.5652, device='cuda:0') tensor(-854.5218, device='cuda:0')


In [40]:
sequences = latent_to_sequence.to_sequence(sampled_unscaled)[-1]

In [41]:
sequences[:3]

['VIMVALPFLHSATRRDTSRSQRLRDDLCARNAPSMRCPRVQFTGPHRCHTINTTYGTCVLMGATLVNRPLSSCYHVGVFETVFTTLVLVESFSLGLEPCNTGDLLDILQGTNCSEMPEQEYSRRDGQK',
 'PSGCGPASSDFVPQAICNVYTAQDPLYLTATGIGAHCFMVISASSSGGIAVTFQSVDAGPSGNCGLDHLEVYDGTRKDWDQAPVLDGRRVLASYCGDEKRGRSLITLNPAAMHFFNAAPGPQAQFHVS',
 'GTNPSIQQVETVTKPTAIFVSDPFSPLNMECVIFIRTRNAPLDNPLADLRVLDFGIDGDCDYDSVSVFDQGYSGFAAPGITLRFCGFIVQSKSYLSTVDIAIVKFYSDVHQYNSARYSFDGLPADQWR']

In [42]:
pdb_strs, raw_outputs = latent_to_structure.to_structure(sampled_unscaled, return_raw_outputs=True, sequences=sequences, batch_size=32)

(Generating structure): 100%|█| 1/1 [00:31<00:00, 31.67


In [43]:
raw_outputs['plddt'].mean(dim=-1).mean(dim=-1)

tensor([45.3554, 52.5439, 49.8820, 39.5031, 50.0599, 51.1328, 53.1777, 44.8893,
        42.4166, 46.8174, 46.2916, 48.0196, 52.2955, 44.6098, 48.9712, 45.8894,
        48.7671, 56.8986, 53.0247, 39.4521, 49.0818, 45.1170, 42.8297, 37.0420,
        39.2021, 39.9285, 41.9869, 45.9710, 41.1934, 36.3155, 43.4090, 44.2516])

In [44]:
print(raw_outputs['plddt'].mean())

tensor(45.8227)


In [45]:
import py3Dmol

# for i in range(len(pdb_strs)):
for i in range(0,10): 
    view = py3Dmol.view(width=600, height=600)
    view.addModelsAsFrames(pdb_strs[i])
    
    # Apply the plDDT color scheme
    # view.setStyle({'cartoon': {'color': {'prop': 'b', 'gradient': 'roygb', 'min': 0, 'max': 100}}})
    view.setStyle({'cartoon': {'color': {'prop': 'b', 'gradient': 'roygb', 'min': 50, 'max': 90}}})
    
    # # Add surface representation with plDDT-based color
    view.addSurface(py3Dmol.VDW, {'opacity': 0.7, 'colorscheme': {'prop': 'b', 'gradient': 'roygb', 'min': 50, 'max': 90}})
    # view.addSurface(py3Dmol.VDW, {'opacity': 0.7, 'colorscheme': {'prop': 'b', 'gradient': 'roygb', 'min': 0, 'max': 100}})

    view.zoomTo()
    view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [46]:
from plaid.evaluation import RITAPerplexity

perplexity_calc = RITAPerplexity(device=device)
perplexities = perplexity_calc.batch_eval(sequences)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [47]:
perplexities

18.984933853149414